In [3]:
import numpy as np
from scipy.stats import skew, kurtosis

def summarize_performance(xs_returns, rf, factor_xs_returns, annualization_factor, txt):
    # Compute total returns
    n_assets = xs_returns.shape[1]
    total_returns = xs_returns + rf * np.ones((xs_returns.shape[0], n_assets))

    # Compute the terminal value of the portfolios to get the geometric mean
    # return per period
    n_periods = xs_returns.shape[0]
    final_pf_val_rf = np.prod(1 + rf)
    final_pf_val_total_ret = np.prod(1 + total_returns)
    geom_avg_rf = 100 * ((final_pf_val_rf ** (annualization_factor / n_periods)) - 1)
    geom_avg_total_return = 100 * ((final_pf_val_total_ret ** (annualization_factor / n_periods)) - 1)
    geom_avg_xs_return = geom_avg_total_return - geom_avg_rf

    # Regress returns on benchmark to get alpha and factor exposures
    X = np.column_stack((np.ones(n_periods), factor_xs_returns))
    b = np.linalg.lstsq(X, xs_returns, rcond=None)[0]
    betas = b[1:, :]

    # Based on the regression estimates, compute the total return on the passive
    # alternative and the annualized alpha
    bm_ret = np.dot(factor_xs_returns, betas) + rf * np.ones((n_periods, n_assets))
    final_pf_val_bm = np.prod(1 + bm_ret)
    geom_avg_bm_return = 100 * ((final_pf_val_bm ** (annualization_factor / n_periods)) - 1)
    alpha_geometric = geom_avg_total_return - geom_avg_bm_return

    # Rescale the returns to be in percentage points
    xs_returns = 100 * xs_returns
    total_returns = 100 * total_returns

    # Compute first three autocorrelations
    ac1 = np.diag(np.corrcoef(xs_returns[:-1, :], xs_returns[1:, :], rowvar=False)[:n_assets, n_assets:])
    ac2 = np.diag(np.corrcoef(xs_returns[:-2, :], xs_returns[2:, :], rowvar=False)[:n_assets, n_assets:])
    ac3 = np.diag(np.corrcoef(xs_returns[:-3, :], xs_returns[3:, :], rowvar=False)[:n_assets, n_assets:])

    # Report the statistics
    print(f'Performance Statistics for {txt}')
    arithm_avg_total_return = annualization_factor * np.mean(total_returns)
    arithm_avg_xs_return = annualization_factor * np.mean(xs_returns)
    std_xs_returns = np.sqrt(annualization_factor) * np.std(xs_returns, ddof=1)
    sharpe_arithmetic = arithm_avg_xs_return / std_xs_returns
    print(f'ArithmAvgTotalReturn: {arithm_avg_total_return}')
    print(f'ArithmAvgXsReturn: {arithm_avg_xs_return}')
    print(f'StdXsReturns: {std_xs_returns}')
    print(f'SharpeArithmetic: {sharpe_arithmetic}')
    print(f'GeomAvgTotalReturn: {geom_avg_total_return}')
    print(f'GeomAvgXsReturn: {geom_avg_xs_return}')
    sharpe_geometric = geom_avg_xs_return / std_xs_returns
    print(f'SharpeGeometric: {sharpe_geometric}')
    print(f'MinXsReturn: {np.min(xs_returns, axis=0)}')
    print(f'MaxXsReturn: {np.max(xs_returns, axis=0)}')
